In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fastparquet

In [2]:
df = pd.read_parquet("meow_df.parquet", engine = 'fastparquet')

In [5]:
tmpst = df.query("""time_id<15""")

In [15]:
df.stock_id = df.stock_id.astype(int)

In [17]:
prob = df.query("""stock_id > 126""")

In [18]:
len(prob)

6475210

In [19]:
len(df)

38382716